In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

# Nehmen Sie an, wir haben drei Gebäcke und ihre Backzeiten
GEBÄCKE = ["Brezeln", "Franzbrötchen", "Laugenstangen"]
BACKZEITEN = [12, 20, 10]

# Definieren Sie die Fitness: Minimieren Sie die gesamte Backzeit
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(len(GEBÄCKE)), len(GEBÄCKE))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalBackzeit(individual):
    """Bewertungsfunktion, die die gesamte Backzeit berechnet"""
    return sum(BACKZEITEN[i] for i in individual),

toolbox.register("evaluate", evalBackzeit)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)  # nur den besten Einzelnen behalten
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", min)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 40, stats=stats, halloffame=hof)

    print("Beste Reihenfolge:", [GEBÄCKE[i] for i in hof[0]])
    print("Gesamte Backzeit:", evalBackzeit(hof[0])[0])


gen	nevals	avg	std	min    	max    
0  	50    	42 	0  	(42.0,)	(42.0,)
1  	38    	42 	0  	(42.0,)	(42.0,)
2  	35    	42 	0  	(42.0,)	(42.0,)
3  	35    	42 	0  	(42.0,)	(42.0,)
4  	38    	42 	0  	(42.0,)	(42.0,)
5  	43    	42 	0  	(42.0,)	(42.0,)
6  	37    	42 	0  	(42.0,)	(42.0,)
7  	38    	42 	0  	(42.0,)	(42.0,)
8  	38    	42 	0  	(42.0,)	(42.0,)
9  	39    	42 	0  	(42.0,)	(42.0,)
10 	40    	42 	0  	(42.0,)	(42.0,)
11 	36    	42 	0  	(42.0,)	(42.0,)
12 	41    	42 	0  	(42.0,)	(42.0,)
13 	34    	42 	0  	(42.0,)	(42.0,)
14 	34    	42 	0  	(42.0,)	(42.0,)
15 	45    	42 	0  	(42.0,)	(42.0,)
16 	40    	42 	0  	(42.0,)	(42.0,)
17 	39    	42 	0  	(42.0,)	(42.0,)
18 	35    	42 	0  	(42.0,)	(42.0,)
19 	42    	42 	0  	(42.0,)	(42.0,)
20 	37    	42 	0  	(42.0,)	(42.0,)
21 	40    	42 	0  	(42.0,)	(42.0,)
22 	43    	42 	0  	(42.0,)	(42.0,)
23 	32    	42 	0  	(42.0,)	(42.0,)
24 	27    	42 	0  	(42.0,)	(42.0,)
25 	35    	42 	0  	(42.0,)	(42.0,)
26 	41    	42 	0  	(42.0,)	(42.0,)
27 	38    	42 	0  	(

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Daten
PRODUKTE = {
    "Brezeln": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"],
    "Franzbrötchen": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"],
    "Laugenstangen": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"]
}

SCHRITT_ZEIT = {
    "Maschine richten": 10,
    "Kneten": 8,
    "In den Ofen": 2,
    "Backen": 15,
    "Aus dem Ofen": 2,
    "Abkühlen lassen": 5
}

UMRÜSTZEIT = 5  # Zeit, um von einem Produkt zum anderen zu wechseln

# Fitness und Individuum
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("product", random.choice, list(PRODUKTE.keys()))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.product, n=10)  # 10 Produkte
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalBackzeit(individual):
    """Bewertungsfunktion, die die Gesamtzeit berechnet"""
    total_time = 0
    prev_product = None
    for product in individual:
        if prev_product and prev_product != product:
            total_time += UMRÜSTZEIT
        for schritt in PRODUKTE[product]:
            total_time += SCHRITT_ZEIT[schritt]
        prev_product = product
    return total_time,

toolbox.register("evaluate", evalBackzeit)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", min)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 40, stats=stats, halloffame=hof)

    print("Beste Reihenfolge:", hof[0])
    print("Gesamte Backzeit:", evalBackzeit(hof[0])[0], "Minuten")


gen	nevals	avg  	std	min     	max     
0  	50    	450.9	6.3	(440.0,)	(465.0,)
1  	38    	446.7	5.25452	(435.0,)	(460.0,)
2  	30    	444  	5.83095	(430.0,)	(455.0,)
3  	43    	441  	5.74456	(430.0,)	(455.0,)
4  	42    	437.9	7.07743	(425.0,)	(455.0,)
5  	32    	434.7	7.02922	(425.0,)	(455.0,)
6  	35    	432.7	6.87095	(425.0,)	(455.0,)
7  	36    	430.3	7.30821	(425.0,)	(455.0,)
8  	34    	427.4	6.01997	(425.0,)	(455.0,)
9  	41    	428.1	5.18556	(425.0,)	(445.0,)
10 	43    	427.3	3.9    	(425.0,)	(440.0,)
11 	31    	429.1	8.16639	(425.0,)	(460.0,)
12 	35    	428.5	5.5    	(425.0,)	(445.0,)
13 	33    	428.9	5.85577	(425.0,)	(445.0,)
14 	40    	427.3	5.9338 	(425.0,)	(450.0,)
15 	35    	426.5	4.03113	(425.0,)	(445.0,)
16 	40    	426.7	4.64866	(425.0,)	(445.0,)
17 	41    	428.4	6.35925	(425.0,)	(445.0,)
18 	33    	427.4	5.76541	(425.0,)	(445.0,)
19 	39    	427  	4.79583	(425.0,)	(445.0,)
20 	39    	427.3	6.1    	(425.0,)	(450.0,)
21 	38    	427.1	5.75239	(425.0,)	(445.0,)
22 	35    	426.8	5.

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Daten
PRODUKTE = {
    "Brezeln": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"],
    "Franzbrötchen": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"],
    "Laugenstangen": ["Maschine richten", "Kneten", "In den Ofen", "Backen", "Aus dem Ofen", "Abkühlen lassen"]
}

SCHRITT_ZEIT = {
    "Maschine richten": 10,
    "Kneten": 8,
    "In den Ofen": 2,
    "Backen": 15,
    "Aus dem Ofen": 2,
    "Abkühlen lassen": 5
}

UMRÜSTZEIT = {
    ('Brezeln', 'Franzbrötchen'): 8,
    ('Brezeln', 'Laugenstangen'): 5,
    ('Franzbrötchen', 'Laugenstangen'): 6,
    # Und umgekehrte Richtungen
    ('Franzbrötchen', 'Brezeln'): 8,
    ('Laugenstangen', 'Brezeln'): 5,
    ('Laugenstangen', 'Franzbrötchen'): 6
}

MITARBEITER = 2

# Fitness und Individuum
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("product", random.choice, list(PRODUKTE.keys()))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.product, n=30)  # 10 Produkte
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def zeigeSchritte(individual):
    """Funktion, um die Schritte für eine gegebene Produktsequenz auszugeben"""
    for product in individual:
        print(f"Produkt: {product}")
        for schritt in PRODUKTE[product]:
            print(f" - {schritt}")

# Beispielaufruf:
ind = toolbox.individual()  # Erzeugen Sie ein zufälliges Individuum
zeigeSchritte(ind)


def evalBackzeit(individual):
    """Bewertungsfunktion, die die Gesamtzeit berechnet"""
    total_time = 0
    prev_product = None
    mitarbeiter_time = [0] * MITARBEITER  # Zeit, die jeder Mitarbeiter verbracht hat

    for product in individual:
        # Umrüstzeit
        if prev_product and prev_product != product:
            total_time += UMRÜSTZEIT.get((prev_product, product), 0)

        # Schritte durchgehen
        for schritt in PRODUKTE[product]:
            min_time = min(mitarbeiter_time)  # Der Mitarbeiter, der am wenigsten beschäftigt ist
            min_idx = mitarbeiter_time.index(min_time)
            mitarbeiter_time[min_idx] += SCHRITT_ZEIT[schritt]

        prev_product = product

    total_time += max(mitarbeiter_time)  # Gesamtzeit ist gleich der am meisten beschäftigten Zeit
    return total_time,

toolbox.register("evaluate", evalBackzeit)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", min)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 40, stats=stats, halloffame=hof)

    print("Beste Reihenfolge:", hof[0])
    print("Gesamte Backzeit:", evalBackzeit(hof[0])[0], "Minuten")


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Produkt: Franzbrötchen
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Franzbrötchen
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Laugenstangen
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Laugenstangen
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Brezeln
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Franzbrötchen
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Brezeln
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Brezeln
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt: Brezeln
 - Maschine richten
 - Kneten
 - In den Ofen
 - Backen
 - Aus dem Ofen
 - Abkühlen lassen
Produkt

In [ ]:
!pip install pyomo
!apt-get install -y glpk-utils




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [ ]:
from pyomo.environ import *

# Daten
PRODUKTE_LIST = ['Brezel', 'Franzbrötchen', 'Laugenstange']
SCHRITTE = ['MaschineRichten', 'Kneten', 'InOfen', 'Backen', 'AusDemOfen', 'AbkühlenLassen']

SCHRITT_ZEIT = {
    'MaschineRichten': 10,
    'Kneten': 15,
    'InOfen': 5,
    'Backen': 20,
    'AusDemOfen': 5,
    'AbkühlenLassen': 30
}

UMRÜSTZEIT = {
    ('Brezel', 'Franzbrötchen'): 15,
    ('Franzbrötchen', 'Brezel'): 15,
    ('Brezel', 'Laugenstange'): 20,
    ('Laugenstange', 'Brezel'): 20,
    ('Franzbrötchen', 'Laugenstange'): 10,
    ('Laugenstange', 'Franzbrötchen'): 10
}

model = ConcreteModel()

# Entscheidungsvariablen
model.X = Var(PRODUKTE_LIST, within=Binary)

# Zielfunktion
def objective_func(model):
    return sum(model.X[p] * sum(SCHRITT_ZEIT[s] for s in SCHRITTE) for p in PRODUKTE_LIST) + sum(UMRÜSTZEIT.get((p1, p2), 0) * model.X[p1] * model.X[p2] for p1 in PRODUKTE_LIST for p2 in PRODUKTE_LIST if p1 != p2)

model.obj = Objective(rule=objective_func, sense=minimize)

# Solver aufrufen
solver = SolverFactory('glpk')
solver.solve(model)

# Ergebnisse ausgeben
for p in PRODUKTE_LIST:
    print(f"{p}: {model.X[p].value}")

print("Objektivwert:", model.obj())


ApplicationError: ignored